In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import pathlib as pl

In [ ]:
import signaturescoring as ssc

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
from statannotations.Annotator import Annotator

In [ ]:
def pretty_ax(ax):
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params(
        axis='both',  
        which='both',      
        bottom=True,     
        top=False,
        left=False,
        labelbottom=True,
        labelleft = True)
    ax.spines["bottom"].set_linewidth(1.5)
    ax.spines["left"].set_linewidth(1.5)

# CAF atlas

In [ ]:
metadata = pd.read_csv("/add/path/here/GSE210347_meta.txt.gz", sep="\t",index_col=0)
adata = sc.read_h5ad("/add/path/here/GSE210347_counts.h5ad")
adata.obs = pd.concat([adata.obs,metadata],axis=1)

In [ ]:
fibadata = adata[adata.obs.celltype=="Fibroblast"].copy()
fibadata.__dict__['_raw'].__dict__['_var'] = fibadata.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})

In [ ]:
fibadata.layers["counts"] = fibadata.X.copy()

sc.pp.normalize_total(fibadata)
sc.pp.log1p(fibadata)

sc.tl.pca(fibadata)

sc.external.pp.harmony_integrate(fibadata, key="SampleID", max_iter_harmony=20)

In [ ]:
sc.pp.neighbors(fibadata, use_rep="X_pca_harmony")

sc.tl.umap(fibadata)

In [ ]:
for sig, genes in fibroblast_markers.items():
    print(sig)
    ssc.score_signature(adata=fibadata,
                        gene_list=list(np.setdiff1d(genes.head(50).names.ravel(),["RUNX1","RUNX2","HGF","ETS1"])), 
                        method="adjusted_neighborhood_scoring", 
                        ctrl_size=150,
                        score_name=f"{sig}_score",use_raw=False)

In [ ]:
fibadata.write_h5ad("/add/path/here/GSE210347_fibroblast_counts.h5ad")

In [ ]:
sc.pl.umap(fibadata, color=['Inflammatory CAF_score', 'Fibroblast_score'] ,ncols=2, frameon=False)

In [ ]:
sc.pl.umap(fibadata, color=['group',"tissue"] ,ncols=2, frameon=False)

In [ ]:
fibadata.obs[['Adipose CAF_score',
       'HGF-CAF_score', 'Inflammatory CAF_score', 'Fibroblast_score']].hist(bins=20)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(4,4))

pairs = [("Adjacent","Normal"),
         ("Adjacent","Tumor"),
         ("Normal","Tumor")]

sns.boxplot(data=fibadata.obs,x="group",
            y="Inflammatory CAF_score", 
            order=["Normal","Adjacent","Tumor"],
            ax=ax)
pretty_ax(ax)
ax.set_xlabel("")

annot = Annotator(
    ax,
    pairs=pairs,
    data=fibadata.obs,x="group",
            y="Inflammatory CAF_score", 
            order=["Normal","Adjacent","Tumor"]
)
annot.configure(
    test="Mann-Whitney",
    loc="inside",
    text_format="star",
    show_test_name=False,
    verbose=2,
    comparisons_correction=None,
    fontsize=10,
)
annot.apply_test()
_, test_results = annot.annotate()

fig.savefig("figures/external/inflamm_CAF_score_type.svg", dpi=200, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(4,4))

pairs = [("Adjacent","Normal"),
         ("Adjacent","Tumor"),
         ("Normal","Tumor")]

sns.boxplot(data=fibadata.obs,x="group",
            y="Fibroblast_score", 
            order=["Normal","Adjacent","Tumor"],
            ax=ax)
pretty_ax(ax)
ax.set_xlabel("")

annot = Annotator(
    ax,
    pairs=pairs,
    data=fibadata.obs,x="group",
            y="Fibroblast_score", 
            order=["Normal","Adjacent","Tumor"]
)
annot.configure(
    test="Mann-Whitney",
    loc="inside",
    text_format="star",
    show_test_name=False,
    verbose=2,
    comparisons_correction=None,
    fontsize=10,
)
annot.apply_test()
_, test_results = annot.annotate()

fig.savefig("figures/external/fibroblast_score_type.svg", dpi=200, bbox_inches="tight")

In [ ]:
TF_expr = pd.DataFrame(fibadata[:,["RUNX1","RUNX2"]].X.copy().toarray(),
                       index=fibadata.obs_names,columns=["RUNX1","RUNX2"])
TF_expr = pd.concat([TF_expr,fibadata.obs[['Adipose CAF_score',
       'HGF-CAF_score', 'Inflammatory CAF_score', 'Fibroblast_score']]],axis=1)

In [ ]:
heatmap_df = TF_expr.corr(method="pearson").loc[['Inflammatory CAF_score', 'Fibroblast_score'],["RUNX1","RUNX2"]]

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,2))
sns.heatmap(data=heatmap_df, annot=heatmap_df, cmap="vlag", center=0, ax=ax)
fig.savefig("figures/external/fibroblast_atlas_heatmap_cNMF_TF_corr.png", dpi=300, bbox_inches="tight")